In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib notebook
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from scipy.spatial.distance import euclidean

import sys
sys.path.append("../src") if "../src" not in sys.path else None # avoid adding multiple relave paths to sys.path

In [3]:
from Agent import Agent, trajectory_generator, batch_trajectory_generator
from Environment import OpenField
from Models import SorscherRNN, UnitPathIntegrator, ExplicitGridCells

from ctimeit import ctimeit # for timing

# Initialise Environment

In [4]:
# OpenField-Environment params
low = (0, 0)
high = (2.2, 2.2)

# Init Environment
env = OpenField(low=low,high=high)

# Initialise Agent / Data generator

In [5]:
# Training data (Agent) params
batch_size = 32
seq_len = 2
angle0 = None # random
p0 = None     # random

# As used by Sorscher
dt = 0.02
sigma = 5.76 * 2
b = 0.13 * 2 * np.pi
mu = 0

# Init training data
btg = batch_trajectory_generator(batch_size, env, seq_len, angle0, p0, dt=dt, sigma=sigma, b=b, mu=mu)

# Format data-generator

In [6]:
def to_one_hot(x):
    """OBS! if max(x,axis=-1) is not unique, then return result is not one-hot, but k-hot"""
    return np.where(x == np.max(x,axis=-1,keepdims=True),1,0)
    
def data_generator(btg):
    while True:
        pos, vel = next(btg)
        pos = brain(pos,activity_model=brain.norm_response)
        init_pos, y_true = pos[:,0], pos[:,1] # "next" pos is label
        vel = vel[:,1] # first velocity (idx 0) is always 0 (initial vel)
        yield (vel,init_pos), y_true 

# Initialise data generator
dg = data_generator(btg)

# Specify and instantiate Model

In [7]:
# Model params
Ng=32
Np=128
weight_decay=None#1e-4
activation=None#"relu"
lr=1e-3 # 1e-3 is default for Adam()

# model init
model = ExplicitGridCells(Ng,Np,weight_decay,activation)

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=lr),
    loss=tf.nn.softmax_cross_entropy_with_logits,
)

# Build (by specifying input_shape) and summarize model
input_shape = [(batch_size, 2), (batch_size, Np)] # velocity-input UNION initial-state
model.build(input_shape)
model.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


NotImplementedError: numpy() is only available when eager execution is enabled.

# Train model

In [ ]:
# train model
epochs = 15
steps_per_epoch = 200
model.fit(x=dg, epochs=epochs, steps_per_epoch=steps_per_epoch)